## 1.导入数据

In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize(96),  
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (0.5,)) 
])

# 加载 Fashion MNIST 数据集
train_dataset = datasets.FashionMNIST(root='./data_FashionMNIST', train=True, transform=transform, download=False)
test_dataset = datasets.FashionMNIST(root='./data_FashionMNIST', train=False, transform=transform, download=False)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

## 2.稠密块

In [5]:
def conv_block(input_channels, num_channels):
    return nn.Sequential(
        nn.BatchNorm2d(input_channels), nn.ReLU(),
        nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1))

class DenseBlock(nn.Module):
    def __init__(self, num_convs, input_channels, num_channels):
        super(DenseBlock, self).__init__()
        layer = []
        for i in range(num_convs):
            layer.append(conv_block(
                num_channels * i + input_channels, num_channels))
        self.net = nn.Sequential(*layer)

    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            # 连接通道维度上每个块的输入和输出
            X = torch.cat((X, Y), dim=1)
        return X

In [6]:
# 测试shape

# 定义一个DenseBlock，输入通道为5，其中包含有3个卷积块，每个输出通道为10，
blk = DenseBlock(3, 5, 10)
# 定义输入X有5个通道
X = torch.randn(1, 5, 6, 6)
Y = blk(X)
# 输出通道数应该为 3*10 + 5 = 35
Y.shape


torch.Size([1, 35, 6, 6])

## 3.过渡层

In [8]:
def transition_block(input_channels, num_channels):
    return nn.Sequential(
        nn.BatchNorm2d(input_channels), nn.ReLU(),
        nn.Conv2d(input_channels, num_channels, kernel_size=1),
        nn.AvgPool2d(kernel_size=2, stride=2))

In [9]:
# 测试shape

# 卷积层减小通道数
# 平均汇聚层 6 -> 3
blk = transition_block(35, 10)
blk(Y).shape

torch.Size([1, 10, 3, 3])

## 4.DenseNet模型

In [11]:
b1 = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
    nn.BatchNorm2d(64), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

# num_channels为当前的通道数
num_channels, growth_rate = 64, 32
num_convs_in_dense_blocks = [4, 4, 4, 4]
blks = []
for i, num_convs in enumerate(num_convs_in_dense_blocks):
    blks.append(DenseBlock(num_convs, num_channels, growth_rate))
    # 上一个稠密块的输出通道数
    num_channels += num_convs * growth_rate
    # 在稠密块之间添加一个转换层，使通道数量减半
    if i != len(num_convs_in_dense_blocks) - 1:
        blks.append(transition_block(num_channels, num_channels // 2))
        num_channels = num_channels // 2

net = nn.Sequential(
    b1, *blks,
    nn.BatchNorm2d(num_channels), nn.ReLU(),
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Linear(num_channels, 10))

## 5.训练模型

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

num_epochs = 10
for epoch in range(num_epochs):
    net.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')

Epoch [1/10], Loss: 0.4273, Train Acc: 84.61%
Epoch [2/10], Loss: 0.2739, Train Acc: 89.94%
Epoch [3/10], Loss: 0.2312, Train Acc: 91.44%
Epoch [4/10], Loss: 0.2097, Train Acc: 92.18%
Epoch [5/10], Loss: 0.1921, Train Acc: 92.77%
Epoch [6/10], Loss: 0.1764, Train Acc: 93.47%
Epoch [7/10], Loss: 0.1608, Train Acc: 94.03%
Epoch [8/10], Loss: 0.1500, Train Acc: 94.47%
Epoch [9/10], Loss: 0.1381, Train Acc: 94.84%
Epoch [10/10], Loss: 0.1253, Train Acc: 95.36%


In [14]:
# 保存整个模型
torch.save(net, 'DenseNet_fashionmnist_model.pth')

## 6.预测

In [16]:
correct = 0
total = 0
net.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_acc = 100 * correct / total
print(f'Test Acc: {test_acc:.2f}%')

Test Acc: 92.41%
